In [14]:
#Function
from math import *
d = 30*60 #minute * 60
W = 0.001
beta = -d ** 2 / log(W)
print(beta)

469038.040455512


In [ ]:
!pip install math
import math
import pandas as pd 
import numpy as np

In [ ]:
Country=pd.read_csv("F:/Combine/UAE_OSRM_Accessibility.csv")
# Drop NaN

# Convert 'duration' to numeric, coercing errors to NaN
Country['duration'] = pd.to_numeric(Country['duration'], errors='coerce')

# Drop rows with NaN values in 'duration' column
Country = Country.dropna(subset=['duration'])
Country['grid_code'] = pd.to_numeric(Country['grid_code'], errors='coerce')

In [ ]:
Country['unique_code'] = Country[['dest_longitude', 'dest_latitude']].apply(lambda x: hash(tuple(x)), axis=1)
Country_acc=Country #backup 

# Apply the exponential decay function
Country_acc['Weight'] = Country_acc['duration'].apply(lambda d: round(math.exp(-d**2/beta), 8))

#compute pop*weight
Country_acc['Pop_W']=Country_acc['grid_code']*Country_acc['Weight'] 

#compute sum(pop_weight)
Countrysum_=Country_acc.groupby(by='unique_code')['Pop_W'].sum().reset_index() 

#join sum(pop weight) to the original table
Country_acc=Country_acc.merge(Countrysum_,on='unique_code')

#supply=1
supply=1
Country_acc = Country_acc.rename(columns={'Pop_W_y': 'Pop_W_S'})#Pop_W_S:POP_WEIGHT_SUM

#compute Rj
Country_acc['supply_demand_ratio']=1/Country_acc.Pop_W_S
Country_acc['supply_demand_ratio'].replace([np.inf, np.nan], 0, inplace=True)

#Rj*W
Country_acc['supply_W']=Country_acc['supply_demand_ratio']*Country_acc.Weight 

#get Ai = sum(Rj*W)
Country_acc['Accessibility'] = Country_acc.groupby('pointid')['supply_W'].transform('sum')
Country_acc_new= Country_acc.drop_duplicates(subset=['pointid'])

# Normalize
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
Country_acc_new['Accessibility_standard'] = scaler.fit_transform(Country_acc_new[['Accessibility']])
Country_acc_new.to_csv("E:/Users/Administrator/Downloads/OSRM_Malawi/Uganda.csv")


<ipython-input-41-d4734834589e>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Country_acc_new['Accessibility_standard'] = scaler.fit_transform(Country_acc_new[['Accessibility']])
